## Download pre-trained Qwen3 model
If not exist in local dir, download from https://www.kaggle.com/models/qwen-lm/qwen-3/transformers/


In [ ]:
from tunix.models.qwen3 import params
from tunix.models.qwen3 import model
from flax import nnx

MODEL_CP_PATH = '<your model download dir>'

config = model.ModelConfig.qwen3_0_6_b()  # pick correponding config based on model version
qwen3 = params.create_model_from_safe_tensors(MODEL_CP_PATH, config)
nnx.display(qwen3)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Qwen3ForCausalLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_CP_PATH)

In [ ]:
from tunix.models.qwen3 import sampler

def templatize(prompts):
  out = []
  for p in prompts:
    out.append(
        tokenizer.apply_chat_template(
          [
              {"role": "user", "content": p},
          ],
          tokenize=False,
          add_generation_prompt=True,
          enable_thinking=True,
      )
    )
  return out

inputs = templatize(
    [
        "which is larger 9.9 or 9.11?",
        "讲几句人话",
        "tell me your name, respond in Chinese",
    ]
)

sampler = sampler.Sampler(qwen3, tokenizer, sampler.CacheConfig(cache_size=256, num_layers=28, num_kv_heads=8, head_dim=128))
out = sampler(inputs, total_generation_steps=128, echo=True)

for t in out.text:
  print(t)
  print('*' * 30)